In [1]:
import logging
import time
from pprint import pprint
import ray
import random
from random import randint
import numpy as np

In [2]:
if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8266', python_version='3.8.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-19_16-52-03_723693_15404/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-19_16-52-03_723693_15404/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-07-19_16-52-03_723693_15404', 'metrics_export_port': 64649, 'gcs_address': '127.0.0.1:63875', 'address': '127.0.0.1:63875', 'node_id': '8ec1346f1e0e07d04020651af4ae01774435a389d0fc02e80db9b4ef'})


In [3]:
print(f"Dashboard url: http://{context.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8266


In [4]:
CALLERS = ["A", "B", "C"]

@ray.remote
class MethodStateCounter:
    def __init__(self):
        self.invokers = {"A": [0], "B": [0], "C": [0]}
    
    def invoke(self, name):
        # pretend to do some work
        time.sleep(0.5)
        
        # compute a random result and update the result
        result = random.randint(5, 25)
        self.invokers[name].append(result)
        return result
        
    def get_invoker_state(self, name):
        return self.invokers[name]
    
    def get_number_of_calls(self, name):
        return len(self.invokers[name])
    
    def get_all_invoker_state(self):
        return self.invokers

In [5]:
actor_instance = MethodStateCounter.remote()

In [6]:
for _ in range(10):
    name = random.choice(CALLERS)
    actor_instance.invoke.remote(name)

#### 1. Get number of times an invoker name was called

In [7]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_number_of_calls.remote(name))
print(f"Number of times: {name} was called: {count}")

Number of times: C was called: 6


#### 2. Get computed values by invoker name

In [8]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_invoker_state.remote(name))
print(f"All values computed by: {name} is: {count}")

All values computed by: A is: [0, 8, 5, 23]


#### 3. Get state of all invokers

In [9]:
print(f"State of all computed values for invokers: {ray.get(actor_instance.get_all_invoker_state.remote())}")

State of all computed values for invokers: {'A': [0, 8, 5, 23], 'B': [0, 11, 9], 'C': [0, 19, 24, 5, 24, 22]}


In [10]:
ray.shutdown()